In [11]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import wandb
from datetime import timedelta
import matplotlib.pyplot as plt
import joblib
from tslearn.neighbors import KNeighborsTimeSeries, KNeighborsTimeSeriesClassifier
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.metrics import dtw
from scripts.NativeGuide.Native_Guide_DBA_lib_1_1 import *

In [12]:
label_file = '../../data/raw/class_labels.csv'
data_dir = '../../data/raw/data/'
cols_to_keep = ['p3_flux_ic', 'p5_flux_ic', 'p7_flux_ic', 'long']

label_map = load_labels(label_file)
X, y, meta = load_mvts_and_labels(data_dir, label_map, cols_to_keep=cols_to_keep, exclude_files=['2005-09-07_14-25.csv'])

X_train, X_val, X_test, y_train, y_val, y_test, meta_train, meta_val, meta_test = train_val_test_split(X, y, meta)
X_train_3D = convert_to_3d_numpy(X_train, cols_to_keep)
X_val_3D   = convert_to_3d_numpy(X_val, cols_to_keep)
X_test_3D  = convert_to_3d_numpy(X_test, cols_to_keep)

model = joblib.load('../../models/KNN_1_TS__classifier_v1.3.pkl')

# query = X_test_3D[0]
# query_label = y_test[0]
# dist, neighbor_indices = native_guide_retrieval_mvts(query, query_label, y_train, X_train_3D)
# plot_query_and_neighbors(query, query_label, X_train_3D, y_train, neighbor_indices, cols_to_keep)
#
# insample_cf = X_train_3D[neighbor_indices[0]]
# generated_cf = get_generated_cf(query, insample_cf, model)
# cf_label = target_mvts(query, model)
# plot_mvts_cf(query, query_label, generated_cf, cf_label, cols_to_keep)
#
# ==================


Experiment 1.2_a

thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]
beta_steps = [0.01, 0.05]

In [16]:
thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]
beta_steps = [0.01, 0.05]
hyperparameter_results = []
# Loop over hyperparameters
for th in thresholds:
    for b_step in beta_steps:
        for i in range(100):  # evaluate on first 100 validation samples
            success_rates = []
            dtw_distances = []
            query = X_val_3D[i]
            pred_label = model.predict(query[np.newaxis, :, :])[0]

            # retrieve unlike neighbor(s)
            _, neighbor_indices = native_guide_retrieval_mvts(query, pred_label, y_train, X_train_3D)
            insample_cf = X_train_3D[neighbor_indices[0]]

            # generate CF
            generated_cf = get_generated_cf(query, insample_cf, model, pred_threshold=th, beta_step=b_step)

            cf_label = model.predict(generated_cf[np.newaxis, :, :])[0]

            # success if CF != predicted label
            success_rates.append(cf_label != pred_label)

            # distance metric (Euclidean between series)
            # dtw_distances.append(dtw(X_test[i].ravel(), generated_cf[i].ravel()))
            for ch in range(X_test_3D.shape[1]):  # for each channel
                dtw_dist = dtw(X_val_3D[i, ch, :], generated_cf[ch, :])
                dtw_distances.append(dtw_dist)

            # Save results
            hyperparameter_results.append({
                "Query_index":i,
                "Query" : query,
                "Pred_label" : pred_label,
                "generated_cf" : generated_cf,
                "cf_label" : cf_label,
                "threshold": th,
                "beta_step": b_step,
                "n_neighbors": 3,
                "success_rate": success_rates,
                "dtw_distances": dtw_distances,
                "avg_dtw_distances": np.mean(dtw_distances)
            })
            print(f"Query Point {i} from Validation set processed.")
        print(f"Search completed for beta value: {b_step}")
    print(f"Search completed for threshold value: {th}")

df_results = pd.DataFrame(hyperparameter_results)
df_results.sort_values(by="success_rate", ascending=False)


Beta used for CF: 0.4300000000000002
Probability for target class: 1.0
Query Point 0 from Validation set processed.
Beta used for CF: 0.5200000000000002
Probability for target class: 1.0
Query Point 1 from Validation set processed.
Beta used for CF: 0.47000000000000025
Probability for target class: 1.0
Query Point 2 from Validation set processed.
Beta used for CF: 0.46000000000000024
Probability for target class: 1.0
Query Point 3 from Validation set processed.
Beta used for CF: 0.38000000000000017
Probability for target class: 1.0
Query Point 4 from Validation set processed.
Beta used for CF: 0.5300000000000002
Probability for target class: 1.0
Query Point 5 from Validation set processed.
Beta used for CF: 0.5400000000000003
Probability for target class: 1.0
Query Point 6 from Validation set processed.
Beta used for CF: 0.35000000000000014
Probability for target class: 1.0
Query Point 7 from Validation set processed.
Beta used for CF: 0.3900000000000002
Probability for target class: 1

,Query_index,Query,Pred_label,generated_cf,cf_label,threshold,beta_step,n_neighbors,success_rate,dtw_distances,avg_dtw_distances
0,0,"[[0.6048, 0.4856, 0.3663999999999999, 0.247199...",0,"[[0.38175899999999996, 0.3150963999999999, 0.2...",1,0.5,0.01,3,[True],"[0.47240472385885385, 0.10897656272300031, 0.0...",0.165943
671,71,"[[2.865, 2.78908, 2.71316, 2.63724, 2.56132, 2...",1,"[[3.0218886000000005, 2.950648, 2.8794074, 2.8...",0,0.8,0.01,3,[True],"[2.8385906397872755, 0.13089490626224937, 0.06...",0.758988
658,58,"[[0.09724, 0.09432, 0.0914, 0.10192, 0.1124399...",0,"[[0.0918928, 0.09180479999999996, 0.0917168000...",1,0.8,0.01,3,[True],"[0.40500981412938647, 0.15917267624614473, 0.0...",0.166024
659,59,"[[0.12648, 0.15586, 0.18524, 0.214619999999999...",0,"[[0.09751868613138687, 0.11479094890510942, 0....",1,0.8,0.01,3,[True],"[0.6618967708432367, 1.1133439051795446, 0.281...",0.514317
660,60,"[[0.0482, 0.0482, 0.0482, 0.0482, 0.04916, 0.0...",0,"[[0.06610400000000002, 0.07623200000000002, 0....",1,0.8,0.01,3,[True],"[0.3395669548860136, 0.15444064251614592, 0.05...",0.137669
...,...,...,...,...,...,...,...,...,...,...,...
338,38,"[[0.06882, 0.06746, 0.0661, 0.0675, 0.0689, 0....",0,"[[0.07799400000000001, 0.088987, 0.09998, 0.11...",1,0.6,0.05,3,[True],"[0.29773795585086255, 0.13549392692017606, 0.0...",0.125453
339,39,"[[0.1838, 0.154, 0.192, 0.2299999999999999, 0....",0,"[[0.18441600000000002, 0.16820319999999994, 0....",1,0.6,0.05,3,[True],"[1.2137841687507187, 1.4166632620698394, 0.339...",0.742529
340,40,"[[0.169368, 0.148812, 0.128256, 0.1077, 0.1095...",0,"[[0.1360608, 0.12491919999999997, 0.1137776, 0...",1,0.6,0.05,3,[True],"[0.24765517771900175, 0.09893658024000622, 0.0...",0.100785
341,41,"[[0.1398, 0.1424, 0.145, 0.1502, 0.1554, 0.160...",0,"[[0.24251399999999998, 0.23377520000000002, 0....",1,0.6,0.05,3,[True],"[1.1120960668726048, 0.9243462599963127, 0.249...",0.571494


In [60]:
df_results = pd.read_csv("./../../reports/hyper_parameter_optimization.csv",sep=",",index_col = 0)

In [62]:
df_results

,Query_index,Query,Pred_label,generated_cf,cf_label,threshold,beta_step,n_neighbors,success_rate,dtw_distances,avg_dtw_distances
0,0,[[6.04800000e-01 4.85600000e-01 3.66400000e-01...,0,[[3.81759000e-01 3.15096400e-01 2.48433800e-01...,1,0.5,0.01,3,[True],"[0.47240472385885385, 0.10897656272300031, 0.0...",0.165943
1,1,[[7.38200000e-02 7.65400000e-02 7.92600000e-02...,0,[[7.98936000e-02 9.21712000e-02 1.04448800e-01...,1,0.5,0.01,3,[True],"[0.2591428006964501, 0.1440962129162318, 0.065...",0.117103
2,2,[[1.54130000e-01 1.41247800e-01 1.28365600e-01...,0,[[1.21873900e-01 1.24963334e-01 1.28052768e-01...,1,0.5,0.01,3,[True],"[0.21526282239918523, 0.10224936736626322, 0.0...",0.097323
3,3,[[6.33000000e-02 5.97000000e-02 5.61000000e-02...,0,[[7.35120000e-02 8.12740000e-02 8.90360000e-02...,1,0.5,0.01,3,[True],"[0.23276040982985058, 0.19958905215837883, 0.1...",0.137762
4,4,[[2.21600000e-01 1.85400000e-01 1.49200000e-01...,0,[[1.70110000e-01 1.48798400e-01 1.27486800e-01...,1,0.5,0.01,3,[True],"[0.2831387510111606, 0.16170701475001, 0.08210...",0.131747
...,...,...,...,...,...,...,...,...,...,...,...
995,95,[[8.90564000e-02 7.97930000e-02 8.59164000e-02...,0,[[8.77260200e-02 8.39721500e-02 8.86810200e-02...,1,0.9,0.05,3,[True],"[0.26156774937337973, 0.12251774664638988, 0.0...",0.114208
996,96,[[2.23000000e-01 2.18400000e-01 2.13800000e-01...,0,[[1.54550000e-01 1.53740000e-01 1.52930000e-01...,1,0.9,0.05,3,[True],"[0.45093403220426836, 0.147199911684756, 0.074...",0.168060
997,97,[[2.91800000e-01 2.92400000e-01 2.93000000e-01...,0,[[2.23710000e-01 2.19360000e-01 2.15010000e-01...,1,0.9,0.05,3,[True],"[1.3066420277719524, 0.3436423555369739, 0.238...",0.472253
998,98,[[1.45800000e-01 1.24000000e-01 1.30800000e-01...,0,[[1.72602000e-01 1.55352400e-01 1.53832800e-01...,1,0.9,0.05,3,[True],"[0.3469776569348667, 0.27141960394176645, 0.15...",0.193600


In [ ]:
Give more weight to the in unlike neighbour